### **Initial Configuration**

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import talib
import torch
import os
from datetime import datetime

from environment import Env
from networks import AdvantageActorCritic
from agent import Agent

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

print("✓ All imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device available: {torch.cuda.is_available() or torch.backends.mps.is_available()}")

✓ All imports successful!
PyTorch version: 2.8.0
Device available: True


In [2]:
# Data configuration
START_DATE = '2010-01-01'
END_DATE = '2024-12-31'
TICKERS = ['VNQ', 'SPY', 'GLD', 'BTC-USD']

# Model configuration
LAG = 5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else 
                      "mps" if torch.backends.mps.is_available() else 
                      "cpu")

# Evaluation configuration
BENCHMARK_WEIGHTS = np.array([0.1, 0.8, 0.05, 0.05, 0])  # VNQ, SPY, GLD, BTC, Cash

print(f"📊 Configuration loaded")
print(f"   Tickers: {TICKERS}")
print(f"   Date range: {START_DATE} to {END_DATE}")
print(f"   Device: {DEVICE}")

📊 Configuration loaded
   Tickers: ['VNQ', 'SPY', 'GLD', 'BTC-USD']
   Date range: 2010-01-01 to 2024-12-31
   Device: mps


### **Data & Features**

In [3]:
print("Downloading price data...")
df = yf.download(TICKERS, start=START_DATE, end=END_DATE)
df = df['Close']
df = df.dropna().copy()

print(f"✓ Downloaded {len(df)} days of data")
print(f"  Date range: {df.index[0]} to {df.index[-1]}")

# Load VIX data
print("\nLoading VIX features...")
vix_df = pd.read_csv('../../data/VIX_CLS_2010_2024.csv')
vix_df['observation_date'] = pd.to_datetime(vix_df['observation_date'])
vix_df = vix_df.set_index('observation_date')
vix_df = vix_df.rename(columns={'VIXCLS': 'VIX'})

vix3m_df = pd.read_csv('../../data/VIX3M_CLS_2010_2024.csv')
vix3m_df['observation_date'] = pd.to_datetime(vix3m_df['observation_date'])
vix3m_df = vix3m_df.set_index('observation_date')
vix3m_df = vix3m_df.rename(columns={'VXVCLS': 'VIX3M'})

vix_combined = vix_df.join(vix3m_df, how='outer')
df = df.join(vix_combined, how='left')
df['VIX'] = df['VIX'].ffill()
df['VIX3M'] = df['VIX3M'].ffill()

# VIX features
df['VIX_normalized'] = (df['VIX'] - 20) / 20

def get_vix_regime(vix):
    if vix < 15:
        return -1.0
    elif vix < 25:
        return 0.0
    else:
        return 1.0

df['VIX_regime'] = df['VIX'].apply(get_vix_regime)
df['VIX_term_structure'] = (df['VIX3M'] - df['VIX']) / df['VIX']
df['VIX_term_structure'] = np.clip(df['VIX_term_structure'], -1, 1)

print("✓ VIX features added")

# RSI features
print("Calculating RSI...")
for ticker in TICKERS:
    df[ticker + '_RSI'] = talib.RSI(df[ticker], timeperiod=14) / 50 - 1

# MACD features
print("Calculating MACD...")
for ticker in TICKERS:
    macd, signal, hist = talib.MACD(df[ticker], fastperiod=12, slowperiod=26, signalperiod=9)
    df[ticker + '_MACD'] = macd / df[ticker] * 10
    df[ticker + '_MACD_Signal'] = signal / df[ticker] * 10

# Volatility features
print("Calculating volatility...")
for ticker in TICKERS:
    returns = df[ticker].pct_change()
    df[ticker + '_volatility'] = returns.rolling(20).std() * np.sqrt(252)
    df[ticker + '_volatility'] = (df[ticker + '_volatility'] - 0.25) / 0.25

df = df.dropna()

print(f"\n✓ Feature engineering complete")
print(f"  Total features: {len([col for col in df.columns if col not in TICKERS + ['VIX', 'VIX3M']])}")
print(f"  Data shape: {df.shape}")

/var/folders/3v/2yw6c8qx0n55dsz1fdnrlrm80000gn/T/ipykernel_69051/694313803.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(TICKERS, start=START_DATE, end=END_DATE)
[*********************100%***********************]  4 of 4 completed

✓ Downloaded 2589 days of data
  Date range: 2014-09-17 00:00:00 to 2024-12-30 00:00:00

Loading VIX features...
✓ VIX features added
Calculating RSI...
Calculating MACD...
Calculating volatility...

✓ Feature engineering complete
  Total features: 19
  Data shape: (2556, 25)


In [4]:
# Split data (80/20)
n_train = int(0.8 * len(df))
df_train = df.iloc[:n_train]
df_test = df.iloc[n_train:]

print("📊 Data Split:")
print(f"   Train: {len(df_train)} days ({df_train.index[0]} to {df_train.index[-1]})")
print(f"   Test:  {len(df_test)} days ({df_test.index[0]} to {df_test.index[-1]})")

# Create test environment
env_test = Env(df_test, TICKERS, lag=LAG)
print(f"\n✓ Test environment created")
print(f"   State dimension: {env_test.states.shape[1] * env_test.lag}")

📊 Data Split:
   Train: 2044 days (2014-11-03 00:00:00 to 2022-12-14 00:00:00)
   Test:  512 days (2022-12-15 00:00:00 to 2024-12-30 00:00:00)
Environment initialized:
  - State features: 19
  - Features per ticker: 4 (RSI, MACD, MACD_Signal, volatility)
  - Market features: 3 (VIX_normalized, VIX_regime, VIX_term_structure)
  - Total state dimension per timestep: 19
  - Lag (temporal window): 5
  - Final input dimension: 95
  - Number of data points: 512

✓ Test environment created
   State dimension: 95


### **Load Train Model**

In [5]:
print("Available models in models/ folder:")
print("="*50)
model_files = [f for f in os.listdir('models') if f.endswith('.pth')]
for i, f in enumerate(model_files, 1):
    file_size = os.path.getsize(os.path.join('models', f)) / 1024
    mod_time = datetime.fromtimestamp(os.path.getmtime(os.path.join('models', f)))
    print(f"{i}. {f}")
    print(f"   Size: {file_size:.1f} KB | Modified: {mod_time.strftime('%Y-%m-%d %H:%M')}")
    print()

Available models in models/ folder:
1. a2c_portfolio_init.pth
   Size: 36.6 KB | Modified: 2025-10-14 12:22

2. a2c_portfolio_1800k_vix.pth
   Size: 54.2 KB | Modified: 2025-10-16 07:07

3. a2c_portfolio_500k_vix.pth
   Size: 54.2 KB | Modified: 2025-10-16 07:07

